In [39]:
# Imports

import pandas as pd
from pandas import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

# Setup

In [40]:
sparql = SPARQLWrapper("http://localhost:7200/repositories/eu")

SPOTIFY_ONTOLOGY = "https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#"


## Query utilities

In [41]:
# Execute a SPARQL query and return a DataFrame
def executeQuery(query):
    global sparql

    # Set the query
    sparql.setQuery(query)

    # Set the result format
    sparql.setReturnFormat(JSON)

    # Execute the query
    results = sparql.query().convert()

    # Create a DataFrame from the returned JSON
    resultDF = json_normalize(results["results"]["bindings"])

    # Remove the datadtypes columns
    resultDF = resultDF[[col for col in resultDF.columns if not col.endswith(
        ".type") and not col.endswith(".datatype")]]

    # Remove .value from column names
    resultDF = resultDF.rename(columns=lambda col: col.replace(".value", ""))

    # Remove the spotify ontology prefix from uris (to get only the ID)
    resultDF = resultDF.applymap(lambda x: str(x).lstrip(SPOTIFY_ONTOLOGY))

    return resultDF


# Queries

In [42]:
executeQuery("""
    PREFIX so: <https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    select distinct ?track ?trackName ?danceability (group_concat(distinct(?genreName); separator=", ") as ?genresNames) (group_concat(distinct(?artistName); separator=", ") as ?artistNames) (group_concat(distinct(?artistRealCompleteName); separator=", ") as ?artistRealNames) where { 
        ?track a so:Track ;
            so:danceability ?danceability ;
            so:name ?trackName .
        ?artist a so:Artist ;
                so:partecipateIn ?track ;
                so:name ?artistName ;
                so:hasGenre ?genre .
        ?genre a so:Genre ;
            so:name ?genreName .
        OPTIONAL {
            ?artistPerson so:isMemberOf ?artist ;
                so:name ?artistRealName ;
                so:surname ?artistRealSurname .
        } .
        BIND(CONCAT(?artistRealName, " ", ?artistRealSurname) AS ?artistRealCompleteName) .
        FILTER (?danceability >= "0.9"^^xsd:float) .
    } group by ?track ?trackName ?danceability order by desc(?danceability)
""")

,track,trackName,danceability,genresNames,artistNames,artistRealNames
0,#4dJvs9kg78ZiHVnNr6UUNJ,Trixma,0.98,"ch hip hop, pop urbaine, rap francais, francoton",Bosh,Bosh
1,#3AEgic4yp82NxMWUjHtxnO,Uno,0.978,cali rap,Ambjaay,
2,#01IQ4aQgOf0Kkq3a273hmO,Funky Friday,0.975,"k hip hop, grime","Fredo, Dave","Marvin William Bailey, David Warren Brubeck"
3,#4d2tgzpNhQO8HIxC4YLnBe,In The Air,0.974,"z reggae, pacific islands pop",L.A.B.,"carbon , nitrogen"
4,#5v7h5vQgCJKZT5vB3I9s3o,In the Air,0.974,"z reggae, pacific islands pop",L.A.B.,"carbon , nitrogen"
...,...,...,...,...,...,...
913,#3wcTpbJfzaQ3MHt22ZJgqm,Messi,0.9,"latino, argentine hip hop, trap argentino",Neo Pistea,Neo Pistea
914,#3g9Mi3kTzzmjBUXN7wnGjI,MotorSport,0.9,", dance pop, pop rap, hip pop, queens hip hop,...","Nicki Minaj, Cardi B, Migos","ka Tanya Maraj Petty, Belcalis Marlenis Almanz..."
915,#3uwDraKGHefaYqUG6cB8QD,Herren på Täppan,0.9,"hip hop, swedish trap pop, swedish gangsta ra...",", Z.E","Mike Kinsella, Z. E"
916,#0IBzvfDLFn9K39wMASeBiQ,"Un Bellakeo (feat. Alexio, Pusho & Juanka El P...",0.9,", reggaeton, trap latino, puerto rican pop, re...","zuna, Pusho, Juanka, Alexio","Juan Carlos Ozuná, Pusho"
